# Inputs & outputs
- **Inputs:** O*NET task statement and rating Excel files in `../final_data/s8_salary_data/` plus BLS wage tables.
- **Outputs:** serialized task vectors, programming-hour weights, and salary reference tables used to build Supplementary Figure S8.

In [ ]:
import pandas as pd
import pickle
from collections import defaultdict
import numpy as np
import json
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import matplotlib.font_manager

# read data

In [ ]:
#import pickle5 as pickle
def save_obj(obj, name, data_path_save = 'obj/'):
    with open(data_path_save + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        

def load_obj(name, data_path_load = 'obj/'):
    with open(data_path_load + name + '.pkl', 'rb') as f:
        return pickle.load(f)


data_path = '../final_data/s8_salary_data/'
df_task_statement = pd.read_excel(data_path + 'Task Statements.xlsx')



## task statements
task_content = {}
job_tasklist = defaultdict(list)
job_content = {}
job_tasktype_list = defaultdict(list)
for soc_id,soc_job, task_id, task, core_sup in zip(df_task_statement['O*NET-SOC Code'],df_task_statement['Title'], df_task_statement['Task ID'],  df_task_statement['Task'], df_task_statement['Task Type']):
    task_content[str(task_id)] = task
    job_tasklist[soc_id].append(str(task_id))
    job_content[soc_id] = soc_job
    if pd.isna(core_sup):
        job_tasktype_list[soc_id].append('Not Available')
    else:
        job_tasktype_list[soc_id].append(core_sup)


## task rating
df_task_rating = pd.read_excel(data_path + 'Task Ratings.xlsx')


category_dict = {str(i+1):i for i in range(7)}
job_task_bool = defaultdict(bool)
job_task_value_ft = {}
job_task_value_im = defaultdict(float)
job_task_value_rt = defaultdict(float)

for soc_id, task_id, scale_id, category, data_value in zip(df_task_rating['O*NET-SOC Code'], df_task_rating['Task ID'], df_task_rating['Scale ID'], df_task_rating['Category'], df_task_rating['Data Value']):
    if scale_id == 'FT':
        if not job_task_bool[(soc_id, str(task_id))]:
            job_task_bool[(soc_id, str(task_id))] = True
            job_task_value_ft[(soc_id, str(task_id))] = [0 for _ in range(7)]

        job_task_value_ft[(soc_id, str(task_id))][category_dict[str(int(category))]] = data_value

    if scale_id == 'IM':
        job_task_value_im[(soc_id, str(task_id))] = data_value

    if scale_id == 'RT':
        job_task_value_rt[(soc_id, str(task_id))] = data_value

print(len(job_task_value_ft), len(job_task_value_im), len(job_task_value_rt))


missing_pairs = []
job_ftlist = defaultdict(list)
job_tasklist_ft = defaultdict(list)

for soc_id, tasks in job_tasklist.items():
    for task_id in tasks:
        if job_task_bool[(soc_id, task_id)]:
            job_ftlist[soc_id].append(job_task_value_ft[(soc_id, task_id)])
            job_tasklist_ft[soc_id].append(task_id)

        else:
            missing_pairs.append([(soc_id, task_id)])


save_obj(job_ftlist,'job_ftvector_list', data_path)
save_obj(job_tasklist_ft,'job_tasklist_ft', data_path)
save_obj(missing_pairs, 'missing_pairs', data_path)
save_obj(job_task_value_rt, 'job_task_value_rt', data_path)
save_obj(job_task_value_im, 'job_task_value_im', data_path)
save_obj(task_content, 'task_content',data_path)
save_obj(job_content, 'job_content',data_path)
save_obj(job_tasktype_list, 'job_tasktype_list',data_path)

# LLM prompts

In [ ]:
# job_tasklist_ft = load_obj('job_tasklist_ft', data_path)
# task_content =load_obj('task_content',data_path)
# job_content = load_obj('job_content',data_path)

# import requests
# from collections import defaultdict

# job_task_programming_score = {}

# # Ollama API 地址
# api_url = ""

# for soc_id, task_list in job_tasklist_ft.items():
#     print(job_content[soc_id])
#     job_task_programming_score[soc_id] = {}
    
#     for t in task_list:
#         question = ''''''
        
#         payload = {
#             "model": "llama3.3:latest",  # 模型名称
#             "prompt":question,  # 输入的提示文本
#             "stream": False,  # 是否流式输出
#             "max_tokens": 20000,  # 限制生成的最大长度
#             "temperature": 0.1,  # 控制生成文本的随机性
#             "top_p": 0.1,  # 控制生成文本的多样性
#         }
        
#         # 发送 POST 请求
#         response = requests.post(api_url, json=payload)
        
#         # 检查响应状态
#         if response.status_code == 200:
#             # 解析响应内容
#             result = response.json()
#             result_temp = result.get("response")
#             print("生成的文本：", result_temp)
#             job_task_programming_score[soc_id][t] = result_temp
#         else:
#             print("请求失败，状态码：", response.status_code)
#             print("错误信息：", response.text)
#             job_task_programming_score[soc_id][t] = 10086

#     break

# get working hours

## examples

In [ ]:
job_ftlist = load_obj('job_ftvector_list', data_path)
job_tasklist_ft = load_obj('job_tasklist_ft', data_path)

hour_vector = [0, 0.02*8, 0.05*8, 0.08*8, 0.10*8, 0.25*8, 0.5*8]

soc_working_time_vector_per_day = {}
for soc_id, ft_matrix in job_ftlist.items():
    ftm = np.array(ft_matrix)
    ftm_normalized = ftm / (np.sum(ftm, axis = 0) + 0.0000000001)
    soc_working_time_vector_per_day[soc_id] = ftm_normalized.dot(np.array(hour_vector).T) / 8

save_obj(soc_working_time_vector_per_day, 'soc_working_time_vector_example_1_ft_normalized', data_path)


job_ftlist = load_obj('job_ftvector_list', data_path)
job_tasklist_ft = load_obj('job_tasklist_ft', data_path)

hour_vector = [0.5, 1, 4, 48, 240, 480, 1920]

soc_working_time_vector = {}
for soc_id, ft_matrix in job_ftlist.items():
    temp = np.array(ft_matrix).dot(np.array(hour_vector).T)
    soc_working_time_vector[soc_id] = temp / np.sum(temp)


save_obj(soc_working_time_vector, 'soc_working_time_vector_example_2_hour_vector', data_path)


job_im_dict = load_obj('job_task_value_im', data_path)
job_tasklist = load_obj('job_tasklist_ft', data_path)

job_im_weighted_time = {}
job_imrank_weighted_time = {}

for soc_id, tasklist in job_tasklist.items():
    im_temp = np.array([job_im_dict[(soc_id, t)] for t in tasklist])
    arr = -im_temp
    temp = arr.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(arr)) + 1
    rank_weight = 2 * len(tasklist) - ranks

    job_im_weighted_time[soc_id] = im_temp / np.sum(im_temp)
    job_imrank_weighted_time[soc_id] = rank_weight / np.sum(rank_weight)

save_obj(job_im_weighted_time, 'job_im_weighted_time', data_path)
save_obj(job_imrank_weighted_time, 'job_imrank_weighted_time', data_path)

## BLS hours

In [ ]:

df_job_salary = pd.read_excel(data_path + 'national_M2024_dl_pure_data.xlsx')
df_job_salary = df_job_salary[df_job_salary['O_GROUP'] == 'detailed']

annual_salary_bls = {}
hour_salary_bls = {}

employment_count_bls = {}

for soc_id, h, a, emp in zip(df_job_salary['OCC_CODE'], df_job_salary['H_MEAN'], df_job_salary['A_MEAN'], df_job_salary['TOT_EMP']):

    employment_count_bls[soc_id] = emp

    if a != '*':
        annual_salary_bls[soc_id] = a

    if h != '*':
        hour_salary_bls[soc_id] = h
        
hour_salary_bls_adjust = {}
hour_salary_bls_adjust.update(hour_salary_bls)
for soc_id, a in annual_salary_bls.items():
    if soc_id not in hour_salary_bls:
        hour_salary_bls_adjust[soc_id] = a / 2080

annual_salary_bls_adjust = {}
annual_salary_bls_adjust.update(annual_salary_bls)
for soc_id, h in hour_salary_bls.items():
    if soc_id not in annual_salary_bls:
        annual_salary_bls_adjust[soc_id] = h * 2080

for soc_id, h in hour_salary_bls_adjust.items():
    if abs(h*2080-annual_salary_bls_adjust[soc_id])>20:
        print(soc_id)

save_obj(annual_salary_bls,'annual_salary_bls', data_path)
save_obj(hour_salary_bls,'hour_salary_bls', data_path)



job_task_programming_score = load_obj("job_task_programming_score_gemini_0_5_adjusted", data_path)
ps_temp = [ts for j,tsl in job_task_programming_score.items() for t, ts in tsl.items()]
print(Counter(ps_temp))

ps_dict = {'0':0, '1':0.0055, '2':0.18, '3':0.38, '4':0.63, '5':0.88}

def calculate_programming_score(soc_working_time_vector):

    job_programming_score_vector = {}
    job_programming_hour = {}

    for soc_id, tasklist in job_tasklist_ft.items():
        job_programming_score_vector[soc_id] = np.array([ps_dict[job_task_programming_score[soc_id][t]] for t in tasklist])
        job_programming_hour[soc_id] = soc_working_time_vector[soc_id].dot(job_programming_score_vector[soc_id])


    job_programming_hour_6digit_temp = defaultdict(list)
    for soc_id, h in job_programming_hour.items():
        job_programming_hour_6digit_temp[soc_id[:7]].append(h)

    job_programming_hour_6digit = {}
    for soc_id, hs in job_programming_hour_6digit_temp.items():
        job_programming_hour_6digit[soc_id] = np.mean(hs)


    job_programming_hour_6digit['13-1020'] = (job_programming_hour_6digit['13-1023'] + job_programming_hour_6digit['13-1021'] + job_programming_hour_6digit['13-1022'])/3
    del job_programming_hour_6digit['13-1023']
    del job_programming_hour_6digit['13-1021']
    del job_programming_hour_6digit['13-1022']

    job_programming_hour_6digit['13-2020'] = job_programming_hour_6digit['13-2023']
    del job_programming_hour_6digit['13-2023']

    job_programming_hour_6digit['29-2010'] = (job_programming_hour_6digit['29-2011'] + job_programming_hour_6digit['29-2012'])/2
    del job_programming_hour_6digit['29-2011']
    del job_programming_hour_6digit['29-2012']


    job_programming_hour_6digit['31-1120'] = (job_programming_hour_6digit['31-1121'] + job_programming_hour_6digit['31-1122'])/2
    del job_programming_hour_6digit['31-1121']
    del job_programming_hour_6digit['31-1122']

    job_programming_hour_6digit['39-7010'] = (job_programming_hour_6digit['39-7011'] + job_programming_hour_6digit['39-7012'])/2
    del job_programming_hour_6digit['39-7011']
    del job_programming_hour_6digit['39-7012']

    job_programming_hour_6digit['47-4090'] = (job_programming_hour_6digit['47-4091'] + job_programming_hour_6digit['47-4099'])/2
    del job_programming_hour_6digit['47-4091']
    del job_programming_hour_6digit['47-4099']

    job_programming_hour_6digit['51-2020'] = (job_programming_hour_6digit['51-2021'] + job_programming_hour_6digit['51-2022'] + job_programming_hour_6digit['51-2023'])/3
    del job_programming_hour_6digit['51-2021']
    del job_programming_hour_6digit['51-2022']
    del job_programming_hour_6digit['51-2023']
    
    job_programming_hour_6digit['51-2090'] = job_programming_hour_6digit['51-2092']
    del job_programming_hour_6digit['51-2092']

    return job_programming_hour_6digit


soc_working_time_vector = load_obj('soc_working_time_vector_example_1_ft_normalized', data_path)
job_programming_hour_6digit = calculate_programming_score(soc_working_time_vector)
save_obj(job_programming_hour_6digit, 'job_programming_hour_6digit_BLS_example1', data_path)

soc_working_time_vector = load_obj('soc_working_time_vector_example_2_hour_vector', data_path)
job_programming_hour_6digit = calculate_programming_score(soc_working_time_vector)
save_obj(job_programming_hour_6digit, 'job_programming_hour_6digit_BLS_example2', data_path)

soc_working_time_vector = load_obj('job_im_weighted_time', data_path)
job_programming_hour_6digit = calculate_programming_score(soc_working_time_vector)
save_obj(job_programming_hour_6digit, 'job_programming_hour_6digit_BLS_im', data_path)

soc_working_time_vector = load_obj('job_imrank_weighted_time', data_path)
job_programming_hour_6digit = calculate_programming_score(soc_working_time_vector)
save_obj(job_programming_hour_6digit, 'job_programming_hour_6digit_BLS_imrank', data_path)


employment_count_bls['51-2020'] = 273300
hour_salary_bls_adjust['51-2020'] = 22.15
annual_salary_bls_adjust['51-2020'] = 46070

save_obj(annual_salary_bls_adjust,'annual_salary_bls_adjust', data_path)
save_obj(hour_salary_bls_adjust,'hour_salary_bls_adjust', data_path)
save_obj(employment_count_bls,'employment_count_bls_adjust', data_path)

# Figure S8 - comparison

In [ ]:
task_content = load_obj('task_content',data_path)
job_content = load_obj('job_content',data_path)
job_task_programming_score2 = load_obj("job_task_programming_score_gemini_0_5_adjusted", data_path)
job_task_programming_score1 = load_obj("job_task_programming_score_gemini_0_5", data_path)
job_task_programming_score3 = load_obj('job_task_programming_score_0_100_from_gemini', data_path)

ps_dict1 = {'1':0, '2':0.125, '3':0.375, '4':0.625, '5':0.875}
ps_dict2 = {'0':0, '1':0.055, '2':0.18, '3':0.38, '4':0.63, '5':0.88}
ps_dict3 = {str(v):int(v)/100 for vv in job_task_programming_score3.values() for v in vv.values()}

job_content_list = []
task_content_list = []
score1 = []
percent1 = []
score2 = []
percent2 = []
score3 = []
percent3 = []
for job, ps in job_task_programming_score1.items():
    for t, s in ps.items():
        if job in job_content and t in task_content:
            job_content_list.append(job_content[job])
            task_content_list.append(task_content[t])
            score1.append(job_task_programming_score1[job][t])
            percent1.append(ps_dict1[job_task_programming_score1[job][t]])
            score2.append(job_task_programming_score2[job][t])
            percent2.append(ps_dict2[job_task_programming_score2[job][t]])
            score3.append(job_task_programming_score3[job][t])
            percent3.append(ps_dict3[job_task_programming_score3[job][t]])

df = pd.DataFrame.from_dict({'job':job_content_list, 'task':task_content_list, 'score1':score1, 'percent1':percent1, 'score2':score2, 'percent2':percent2, 'score3':score3, 'percent3':percent3 })
df.to_csv(data_path + 'task_score_percent.csv')

job_ftlist = load_obj('job_ftvector_list', data_path)
job_tasklist_ft = load_obj('job_tasklist_ft', data_path)

job_task_programming_score1 = load_obj("job_task_programming_score_gemini_0_5", data_path)
ps_dict1 = {'1':0, '2':0.125, '3':0.375, '4':0.625, '5':0.875}


job_task_programming_score2 = load_obj("job_task_programming_score_gemini_0_5_adjusted", data_path)
ps_dict2 = {'0':0, '1':0.055, '2':0.18, '3':0.38, '4':0.63, '5':0.88}


job_task_programming_score3 = load_obj('job_task_programming_score_0_100_from_gemini', data_path)
ps_dict3 = {str(v):int(v)/100 for vv in job_task_programming_score3.values() for v in vv.values()}


df_skills = pd.read_excel(data_path + 'db_29_3_excel/Skills.xlsx')
df_abilities = pd.read_excel(data_path + 'db_29_3_excel/Abilities.xlsx')
df_knowledges = pd.read_excel(data_path + 'db_29_3_excel/Knowledge.xlsx')

tasks_std = list(set(df_skills['Element ID'])) + list(set(df_abilities['Element ID'])) + list(set(df_knowledges['Element ID']))
tasks_dict_std  = {s:i for i,s in enumerate(tasks_std)}

job_list = list(set(df_skills['O*NET-SOC Code']))
job_task_im_vector = {j:np.zeros(len(tasks_std)) for j in job_list}
job_task_lv_vector = {j:np.zeros(len(tasks_std)) for j in job_list}

for o, s, imlv, v in zip(df_skills['O*NET-SOC Code'], df_skills['Element ID'], df_skills['Scale ID'], df_skills['Data Value']):
    if imlv == 'IM':
        job_task_im_vector[o][tasks_dict_std[s]] = v

    if imlv == 'LV':
        job_task_lv_vector[o][tasks_dict_std[s]] = v


for o, s, imlv, v in zip(df_abilities['O*NET-SOC Code'], df_abilities['Element ID'], df_abilities['Scale ID'], df_abilities['Data Value']):
    if imlv == 'IM':
        job_task_im_vector[o][tasks_dict_std[s]] = v

    if imlv == 'LV':
        job_task_lv_vector[o][tasks_dict_std[s]] = v


for o, s, imlv, v in zip(df_knowledges['O*NET-SOC Code'], df_knowledges['Element ID'], df_knowledges['Scale ID'], df_knowledges['Data Value']):
    if imlv == 'IM':
        job_task_im_vector[o][tasks_dict_std[s]] = v

    if imlv == 'LV':
        job_task_lv_vector[o][tasks_dict_std[s]] = v

programming_id = '2.B.3.e'

job_task_ps_im = {j: v[tasks_dict_std[programming_id]] / np.sum(v) for j,v in job_task_im_vector.items()}
job_task_ps_lv = {j: v[tasks_dict_std[programming_id]] / np.sum(v) for j,v in job_task_lv_vector.items()}

df_skills = pd.read_excel(data_path + 'db_29_3_excel/Skills.xlsx')
df_abilities = pd.read_excel(data_path + 'db_29_3_excel/Abilities.xlsx')
df_knowledges = pd.read_excel(data_path + 'db_29_3_excel/Knowledge.xlsx')

tasks_std = list(set(df_skills['Element ID'])) + list(set(df_abilities['Element ID'])) + list(set(df_knowledges['Element ID']))
tasks_dict_std  = {s:i for i,s in enumerate(tasks_std)}

job_list = list(set(df_skills['O*NET-SOC Code']))
job_task_im_vector = {j:np.zeros(len(tasks_std)) for j in job_list}
job_task_lv_vector = {j:np.zeros(len(tasks_std)) for j in job_list}

for o, s, imlv, v in zip(df_skills['O*NET-SOC Code'], df_skills['Element ID'], df_skills['Scale ID'], df_skills['Data Value']):
    if imlv == 'IM':
        job_task_im_vector[o][tasks_dict_std[s]] = v

    if imlv == 'LV':
        job_task_lv_vector[o][tasks_dict_std[s]] = v


for o, s, imlv, v in zip(df_abilities['O*NET-SOC Code'], df_abilities['Element ID'], df_abilities['Scale ID'], df_abilities['Data Value']):
    if imlv == 'IM':
        job_task_im_vector[o][tasks_dict_std[s]] = v

    if imlv == 'LV':
        job_task_lv_vector[o][tasks_dict_std[s]] = v


for o, s, imlv, v in zip(df_knowledges['O*NET-SOC Code'], df_knowledges['Element ID'], df_knowledges['Scale ID'], df_knowledges['Data Value']):
    if imlv == 'IM':
        job_task_im_vector[o][tasks_dict_std[s]] = v

    if imlv == 'LV':
        job_task_lv_vector[o][tasks_dict_std[s]] = v

programming_id = '2.B.3.e'

job_task_ps_im = {j: v[tasks_dict_std[programming_id]] / np.sum(v) for j,v in job_task_im_vector.items()}
job_task_ps_lv = {j: v[tasks_dict_std[programming_id]] / np.sum(v) for j,v in job_task_lv_vector.items()}



soc_working_time_vector = load_obj('soc_working_time_vector_example_1_ft_normalized', data_path)

job_programming_hour1 = {}
for soc_id, tasklist in job_tasklist_ft.items():
    job_programming_hour1[soc_id] = soc_working_time_vector[soc_id].dot(np.array([ps_dict1[job_task_programming_score1[soc_id][t]] for t in tasklist]))

job_programming_hour2 = {}
for soc_id, tasklist in job_tasklist_ft.items():
    job_programming_hour2[soc_id] = soc_working_time_vector[soc_id].dot(np.array([ps_dict2[job_task_programming_score2[soc_id][t]] for t in tasklist]))

job_programming_hour3 = {}
for soc_id, tasklist in job_tasklist_ft.items():
    job_programming_hour3[soc_id] = soc_working_time_vector[soc_id].dot(np.array([ps_dict3[job_task_programming_score3[soc_id][t]] for t in tasklist]))

common_jobs = list(set(job_task_ps_im.keys()).intersection(job_programming_hour1.keys()))

y_ex1 = [job_task_ps_im[j] for j in common_jobs]
x1_ex1 = [job_programming_hour1[j] for j in common_jobs]
x2_ex1 = [job_programming_hour2[j] for j in common_jobs]
x3_ex1 = [job_programming_hour3[j] for j in common_jobs]



soc_working_time_vector = load_obj('soc_working_time_vector_example_2_hour_vector', data_path)

job_programming_hour1 = {}
for soc_id, tasklist in job_tasklist_ft.items():
    job_programming_hour1[soc_id] = soc_working_time_vector[soc_id].dot(np.array([ps_dict1[job_task_programming_score1[soc_id][t]] for t in tasklist]))

job_programming_hour2 = {}
for soc_id, tasklist in job_tasklist_ft.items():
    job_programming_hour2[soc_id] = soc_working_time_vector[soc_id].dot(np.array([ps_dict2[job_task_programming_score2[soc_id][t]] for t in tasklist]))

job_programming_hour3 = {}
for soc_id, tasklist in job_tasklist_ft.items():
    job_programming_hour3[soc_id] = soc_working_time_vector[soc_id].dot(np.array([ps_dict3[job_task_programming_score3[soc_id][t]] for t in tasklist]))

common_jobs = list(set(job_task_ps_im.keys()).intersection(job_programming_hour1.keys()))

y_ex2 = [job_task_ps_im[j] for j in common_jobs]
x1_ex2 = [job_programming_hour1[j] for j in common_jobs]
x2_ex2 = [job_programming_hour2[j] for j in common_jobs]
x3_ex2 = [job_programming_hour3[j] for j in common_jobs]



# 设置 seaborn 的主题
#sns.set_theme(style="whitegrid")

xl = [x1_ex1,x2_ex1,x3_ex1,x1_ex2,x2_ex2,x3_ex2]
yl = [y_ex1, y_ex1, y_ex1, y_ex2, y_ex2, y_ex2]
x_labels = ["programming share (prompt 1)","programming share (prompt 2)","programming share (prompt 3)","programming share (prompt 1)","programming share (prompt 2)","programming share (prompt 3)"]

ccc = 0
for x,y,l in zip(xl, yl, x_labels):
    ccc += 1

    # 1. 计算皮尔逊相关系数和 p-value
    corr_coefficient, p_value = pearsonr(x, y)

    # 2. 绘制散点图
    plt.figure(figsize=(8, 6))

    font_dirs = [f'{data_path}helvetica-255/', ]
    font_files = matplotlib.font_manager.findSystemFonts(fontpaths=font_dirs)
    for ff in font_files:
        matplotlib.font_manager.fontManager.addfont(ff)

    plt.rcParams['font.family'] = ['Helvetica']
    font1 = {'family':'helvetica',
            'weight' : 'normal',
            'size'   : 23,
    }

    scatter_plot = sns.scatterplot(x=x, y=y, s=70) # s是点的大小

    text_str = f'correlation: {corr_coefficient:.2f}'
    plt.text(0.05, 0.95, text_str, transform=plt.gca().transAxes,
            fontdict=font1, verticalalignment='top',)

    # 添加标题和标签

    plt.xlabel(l, fontdict=font1, fontsize = 27)
    plt.ylabel("'programming' importance", fontdict=font1, fontsize = 27)
    plt.tick_params(labelsize=12)
    #plt.legend()
    plt.rcParams['font.weight'] = 'bold'
    plt.rcParams['axes.labelweight'] = 'bold'
    ax=plt.gca();#获得坐标轴的句柄
    ax.spines['bottom'].set_linewidth(2);###设置底部坐标轴的粗细
    ax.spines['left'].set_linewidth(2);####设置左边坐标轴的粗细
    ax.spines['right'].set_linewidth(2);###设置右边坐标轴的粗细
    ax.spines['top'].set_linewidth(2);####设置上部坐标轴的粗细
    # 显示图像
    plt.xticks([i*0.1 for i in range(9)],[round(i*0.1,1) for i in range(9)])
    plt.yticks([i*0.004 for i in range(5)],[i*0.004 for i in range(5)])